In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

def cost(i,j):
    i,j = get(i), get(j)
    return distmat[i,j]
def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
loops = json.load(open('./loops.json'))
loops

[[0, 1, 45, 101, 44, 16, 0],
 [0, 59, 0],
 [0, 102, 47, 7, 48, 0],
 [0, 12, 123, 0],
 [0, 88, 86, 2, 0],
 [0, 132, 66, 89, 95, 19, 0],
 [0, 46, 61, 60, 3, 0],
 [0, 51, 49, 0],
 [0, 69, 120, 4, 151, 0],
 [0, 64, 118, 77, 126, 0],
 [0, 111, 34, 125, 0],
 [0, 127, 0],
 [0, 172, 5, 0],
 [0, 40, 108, 35, 6, 0],
 [0, 115, 26, 152, 0],
 [0, 153, 169, 128, 0],
 [0, 57, 72, 98, 97, 0],
 [0, 92, 0],
 [0, 164, 27, 8, 71, 163, 0],
 [0, 32, 0],
 [0, 107, 135, 131, 106, 0],
 [0, 133, 9, 113, 167, 0],
 [0, 134, 0],
 [0, 10, 17, 56, 63, 0],
 [0, 55, 180, 0],
 [0, 145, 122, 21, 62, 0],
 [0, 75, 171, 31, 136, 0],
 [0, 84, 137, 0],
 [0, 110, 11, 81, 138, 0],
 [0, 13, 74, 20, 43, 0],
 [0, 85, 161, 0],
 [0, 160, 0],
 [0, 162, 25, 99, 0],
 [0, 42, 73, 0],
 [0, 157, 14, 78, 0],
 [0, 158, 177, 0],
 [0, 159, 94, 96, 0],
 [0, 154, 117, 0],
 [0, 156, 82, 79, 104, 0],
 [0, 176, 15, 155, 165, 0],
 [0, 114, 38, 116, 50, 0],
 [0, 146, 175, 0],
 [0, 147, 18, 170, 174, 0],
 [0, 22, 178, 0],
 [0, 93, 28, 33, 130, 0],
 

In [3]:
h = np.zeros(len(loops), dtype=float)
for i, loop in enumerate(loops):
    for j in range(0, len(loop)-1):
        arc = loop[j], loop[j+1]
        h[i] += time(*arc)
h = h / 3600

In [4]:
D = range(16)
L = range(0, len(loops))
T = range(9)

m = gb.Model()
z = m.addVars(D, vtype=GRB.BINARY, name='z')
y = m.addVars(L,T,D,vtype=GRB.BINARY, name='y')

m.setObjective(z.sum('*'), GRB.MINIMIZE)
m.addConstrs((y.sum(l,'*', '*') == 1 for l in L))
m.addConstrs((
    sum([h[l]*y[l,t,d] for l in L]) <= 8 for t in T for d in D
))
m.addConstrs((y[l,t,d] <= z[d] for l in L for t in T for d in D))
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 8844 rows, 8656 columns and 34560 nonzeros
Model fingerprint: 0xcd90b312
Variable types: 0 continuous, 8656 integer (8656 binary)
Coefficient statistics:
  Matrix range     [3e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 16.0000000
Presolve time: 0.18s
Presolved: 8844 rows, 8656 columns, 34560 nonzeros
Variable types: 0 continuous, 8656 integer (8656 binary)

Root relaxation: objective 1.111111e-01, 8659 iterations, 0.41 seconds (0.16 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.11111    0 1082   16

In [5]:
y

{(0, 0, 0): <gurobi.Var y[0,0,0] (value -0.0)>,
 (0, 0, 1): <gurobi.Var y[0,0,1] (value -0.0)>,
 (0, 0, 2): <gurobi.Var y[0,0,2] (value -0.0)>,
 (0, 0, 3): <gurobi.Var y[0,0,3] (value -0.0)>,
 (0, 0, 4): <gurobi.Var y[0,0,4] (value -0.0)>,
 (0, 0, 5): <gurobi.Var y[0,0,5] (value -0.0)>,
 (0, 0, 6): <gurobi.Var y[0,0,6] (value -0.0)>,
 (0, 0, 7): <gurobi.Var y[0,0,7] (value -0.0)>,
 (0, 0, 8): <gurobi.Var y[0,0,8] (value -0.0)>,
 (0, 0, 9): <gurobi.Var y[0,0,9] (value -0.0)>,
 (0, 0, 10): <gurobi.Var y[0,0,10] (value -0.0)>,
 (0, 0, 11): <gurobi.Var y[0,0,11] (value -0.0)>,
 (0, 0, 12): <gurobi.Var y[0,0,12] (value -0.0)>,
 (0, 0, 13): <gurobi.Var y[0,0,13] (value -0.0)>,
 (0, 0, 14): <gurobi.Var y[0,0,14] (value -0.0)>,
 (0, 0, 15): <gurobi.Var y[0,0,15] (value -0.0)>,
 (0, 1, 0): <gurobi.Var y[0,1,0] (value -0.0)>,
 (0, 1, 1): <gurobi.Var y[0,1,1] (value -0.0)>,
 (0, 1, 2): <gurobi.Var y[0,1,2] (value -0.0)>,
 (0, 1, 3): <gurobi.Var y[0,1,3] (value -0.0)>,
 (0, 1, 4): <gurobi.Var y[0,

In [6]:
Loops = []
for loop in loops:
    L = []
    for l in loop:
        if l in repeats:
            L.append(repeats[l]['map'])
        else:
            L.append(l)
    Loops.append(L)

In [7]:
Loops

[[0, 1, 45, 101, 44, 16, 0],
 [0, 59, 0],
 [0, 102, 47, 7, 48, 0],
 [0, 12, 123, 0],
 [0, 88, 86, 2, 0],
 [0, 12, 66, 89, 95, 19, 0],
 [0, 46, 61, 60, 3, 0],
 [0, 51, 49, 0],
 [0, 69, 120, 4, 40, 0],
 [0, 64, 118, 77, 5, 0],
 [0, 111, 34, 5, 0],
 [0, 5, 0],
 [0, 111, 5, 0],
 [0, 40, 108, 35, 6, 0],
 [0, 115, 26, 40, 0],
 [0, 40, 108, 6, 0],
 [0, 57, 72, 98, 97, 0],
 [0, 92, 0],
 [0, 98, 27, 8, 71, 97, 0],
 [0, 32, 0],
 [0, 107, 32, 9, 106, 0],
 [0, 32, 9, 113, 106, 0],
 [0, 32, 0],
 [0, 10, 17, 56, 63, 0],
 [0, 55, 122, 0],
 [0, 38, 122, 21, 62, 0],
 [0, 75, 110, 31, 33, 0],
 [0, 84, 33, 0],
 [0, 110, 11, 81, 33, 0],
 [0, 13, 74, 20, 43, 0],
 [0, 85, 43, 0],
 [0, 43, 0],
 [0, 43, 25, 99, 0],
 [0, 42, 73, 0],
 [0, 42, 14, 78, 0],
 [0, 42, 118, 0],
 [0, 42, 94, 96, 0],
 [0, 41, 117, 0],
 [0, 41, 82, 79, 104, 0],
 [0, 117, 15, 41, 104, 0],
 [0, 114, 38, 116, 50, 0],
 [0, 38, 116, 0],
 [0, 38, 18, 109, 114, 0],
 [0, 22, 119, 0],
 [0, 93, 28, 33, 6, 0],
 [0, 6, 103, 87, 119, 0],
 [0, 100, 7